In [164]:
from pathlib import Path
import duckdb
import pandas as pd

data_root = Path("D:\\데이터\\건축데이터 건축허브 개방데이터")

path_건축_DB_to = data_root / "건축인허가_2025년_02월.db"
path_주택_DB_to = data_root / "주택인허가_2025년_02월.db"

건축허브에서 제공하는 건축인허가, 주택인허가 데이터를 활용하여 분석

2025년 2월 이전 데이터는 위반건축물 등 일부 건축물이 빠져있는 문제가 있어 부득이 2025년 2월 말 데이터로 2015~2024년 인허가 현황을 집계

In [165]:
# Open two DuckDB connections
# Define the database paths
DB_spec = {
    path_건축_DB_to.stem: (
        con_건축 := duckdb.connect(database=path_건축_DB_to, read_only=True)
    ),
    path_주택_DB_to.stem: (
        con_주택 := duckdb.connect(database=path_주택_DB_to, read_only=True)
    ),
}

for db_name, con in DB_spec.items():
    print(f"Database: {db_name}")
    # print the list of tables in the database
    tables = con.execute("SHOW TABLES").fetchall()
    print("Tables in the database:")
    for table in tables:
        print(table[0])
    print()

for db_name, con in DB_spec.items():
    print(f"Database: {db_name}")
    # show columns and heads of the tables
    tables = con.execute("SHOW TABLES").fetchall()
    for table in tables:
        table_name = table[0]
        df = con.sql(f"SELECT * FROM {table_name}").limit(5).df()
        print(f"Columns of {table_name}:")
        print(df.columns)
        print(f"Head of {table_name}:")
        print(df)
        print()

Database: 건축인허가_2025년_02월
Tables in the database:
기본개요
동별개요
층별개요

Database: 주택인허가_2025년_02월
Tables in the database:
기본개요
동별개요
층별개요
행위개요

Database: 건축인허가_2025년_02월
Columns of 기본개요:
Index(['관리_허가대장_PK', '대지_위치', '건물_명', '시군구_코드', '법정동_코드', '대지_구분_코드', '번', '지',
       '특수지_명', '블록', '로트', '지목_코드_명', '지역_코드_명', '지구_코드_명', '구역_코드_명',
       '지목_코드', '지역_코드', '지구_코드', '구역_코드', '건축_구분_코드', '건축_구분_코드_명',
       '대지_면적(㎡)', '건축_면적(㎡)', '건폐_율(%)', '연면적(㎡)', '용적_률_산정_연면적(㎡)',
       '용적_률(%)', '주_건축물_수', '부속_건축물_동_수', '주_용도_코드', '주_용도_코드_명', '세대_수(세대)',
       '호_수(호)', '가구_수(가구)', '총_주차_수', '착공_예정_일', '착공_연기_일', '실제_착공_일',
       '건축_허가_일', '사용승인_일', '생성_일자'],
      dtype='object')
Head of 기본개요:
               관리_허가대장_PK                         대지_위치              건물_명  \
0  1000000000000000045934  충청남도 천안시 동남구 성남면 석곡리 376-8번지              None   
1  1000000000000000001810         인천광역시 중구 운서동 2765-3번지        운서동 2765-3   
2           1063100083397        인천광역시 옹진군 백령면 가을리 16번지              None

In [166]:
# Count the number of records in each table and display them in a DataFrame
# Create a list to store the table names and their record counts
table_counts = []

# Iterate through the DBs and the tables and count the records
for db_name, con in DB_spec.items():
    tables = con.execute("SHOW TABLES").fetchall()
    for table in tables:
        table_name = table[0]
        count = con.execute(f"SELECT COUNT(*) FROM {table_name}").fetchone()[0]  # type: ignore
        table_counts.append(
            {"DB name": db_name, "Table Name": table_name, "Record Count": count}
        )

# Convert the list to a DataFrame
record_counts_df = pd.DataFrame(table_counts)

# Display the DataFrame
display(record_counts_df)

,DB name,Table Name,Record Count
0,건축인허가_2025년_02월,기본개요,6207241
1,건축인허가_2025년_02월,동별개요,8060261
2,건축인허가_2025년_02월,층별개요,14361599
3,주택인허가_2025년_02월,기본개요,306182
4,주택인허가_2025년_02월,동별개요,367273
5,주택인허가_2025년_02월,층별개요,3152579
6,주택인허가_2025년_02월,행위개요,289370


각 DB별, 테이블별 레코드 수 확인

In [167]:
import duckdb
import pandas as pd

# ──────────── 2) 컬럼 정의 ────────────
columns_건축_일자 = [
    "건축_허가_일",
    "착공_예정_일",
    "착공_연기_일",
    "실제_착공_일",
    "사용승인_일",
]

columns_주택_일자 = [
    "승인_일",
    "착공_예정_일",
    "착공_일",
    "사용_검사_예정_일",
    "사용_검사_일",
]


# ──────────── 3) 집계 함수 정의 ────────────
# 2015년부터 2024년까지 YYYYMMDD 형식의 날짜 범위에 해당하는
# 각 컬럼의 건수를 세는 함수 정의
def count_between_dates(
    connection, table_name, column_names, lower="20150101", upper="20241231"
):
    """
    connection : duckdb.Connection
    table_name : str (예: '기본개요')
    column_names : iterable of str
    lower, upper : str (YYYYMMDD 형식)

    반환: pandas.DataFrame with columns ['컬럼명', '조건_이상_이하_건수']
    """
    results = []
    for col in column_names:
        # SQL: 문자열 비교로도 YYYYMMDD 범위 비교가 가능
        sql = f"""
        SELECT
            COUNT(*) AS cnt
        FROM {table_name}
        WHERE "{col}" >= '{lower}'
          AND "{col}" <= '{upper}'
        """
        res = connection.execute(sql).fetchone()  # 집계 결과가 (cnt,) 형태로 돌아옴
        results.append({"컬럼명": col, "레코드 수": res[0]})

    return pd.DataFrame(results)


# ──────────── 4) 건축: 카운트 수행 ────────────
df_건축_결과 = count_between_dates(
    connection=con_건축,
    table_name="기본개요",
    column_names=columns_건축_일자,
    lower="20150101",
    upper="20241231",
)

# ──────────── 5) 주택: 카운트 수행 ────────────
df_주택_결과 = count_between_dates(
    connection=con_주택,
    table_name="기본개요",
    column_names=columns_주택_일자,
    lower="20150101",
    upper="20241231",
)

# ──────────── 6) 결과 출력 ────────────
print("── 건축DB 기본개요: 각 컬럼별 ’20150101‘ 이상, ’20241231‘ 이하 건수 ──")
display(df_건축_결과)

print("\n── 주택DB 기본개요: 각 컬럼별 ’20150101‘ 이상, ’20241231‘ 이하 건수 ──")
display(df_주택_결과)


── 건축DB 기본개요: 각 컬럼별 ’20150101‘ 이상, ’20241231‘ 이하 건수 ──


,컬럼명,레코드 수
0,건축_허가_일,2836765
1,착공_예정_일,1295260
2,착공_연기_일,85951
3,실제_착공_일,1293842
4,사용승인_일,1238884



── 주택DB 기본개요: 각 컬럼별 ’20150101‘ 이상, ’20241231‘ 이하 건수 ──


,컬럼명,레코드 수
0,승인_일,189594
1,착공_예정_일,79073
2,착공_일,37746
3,사용_검사_예정_일,84044
4,사용_검사_일,78184


실제 착공일(건축: 실제 착공 일, 주택: 착공 일) 건수가 착공예정일보다 적음. 준공일(건축: 사용승인 일, 주택: 사용 검사 일)이 있는 경우, 착공을 했다고 보고, 실제 착공일 → 착공 연기일 → 착공 예정일 순으로 착공일 추정

In [168]:
# ─── 건축DB: 일부 컬럼 선택 ───
# 착공일 추정 컬럼 추가: 사용승인일이 있으면 (실제 착공일 → 착공 연기일 → 착공 예정일 → 사용승인일), 없으면 실제 착공일(그대로, null 허용)
sql_건축_기본일부 = """
SELECT
    "관리_허가대장_PK",
    "시군구_코드",
    주_용도_코드,
    주_용도_코드_명,
    "건축_구분_코드",
    "건축_구분_코드_명",
    "연면적(㎡)",
    "건축_허가_일",
    "착공_예정_일",
    "착공_연기_일",
    "실제_착공_일",
    "사용승인_일",
    CASE
        WHEN "사용승인_일" IS NOT NULL AND "사용승인_일" != ''
            THEN COALESCE(NULLIF("실제_착공_일", ''), NULLIF("착공_연기_일", ''), NULLIF("착공_예정_일", ''))
        ELSE "실제_착공_일"
    END AS 추정_착공일
FROM 기본개요;
"""

# 쿼리 실행 후 DataFrame으로 가져오기
건축_기본일부 = con_건축.sql(sql_건축_기본일부)

# 결과 확인 (예시)
print(건축_기본일부.limit(5))

# 실제착공일은 NULL이고 사용승인일은 NOT NULL인 데이터 5개를 SQL로 추출
query = """
SELECT *
FROM 건축_기본일부
WHERE ("실제_착공_일" IS NULL OR "실제_착공_일" = '')
    AND ("사용승인_일" IS NOT NULL AND "사용승인_일" != '')
LIMIT 5
"""
result = con_건축.sql(query)
print(result)


┌────────────────────────┬─────────────┬──────────────┬───────────────────┬────────────────┬───────────────────┬───────────────┬──────────────┬──────────────┬──────────────┬──────────────┬─────────────┬─────────────┐
│    관리_허가대장_PK    │ 시군구_코드 │ 주_용도_코드 │  주_용도_코드_명  │ 건축_구분_코드 │ 건축_구분_코드_명 │  연면적(㎡)   │ 건축_허가_일 │ 착공_예정_일 │ 착공_연기_일 │ 실제_착공_일 │ 사용승인_일 │ 추정_착공일 │
│        varchar         │   varchar   │   varchar    │      varchar      │    varchar     │      varchar      │ decimal(18,3) │   varchar    │   varchar    │   varchar    │   varchar    │   varchar   │   varchar   │
├────────────────────────┼─────────────┼──────────────┼───────────────────┼────────────────┼───────────────────┼───────────────┼──────────────┼──────────────┼──────────────┼──────────────┼─────────────┼─────────────┤
│ 1000000000000000045934 │ 44131       │ NULL         │ NULL              │ NULL           │ NULL              │        45.000 │ 20220928     │ NULL         │ NULL         │ NULL         │ NULL        

### 건축행위별 데이터 검증

In [169]:
# 사용승인일이 정상인 데이터를 건축행위별로 세기
query = """
SELECT
    "건축_구분_코드",
    "건축_구분_코드_명",
    COUNT(*) AS count
FROM 건축_기본일부
WHERE "사용승인_일" IS NOT NULL
  AND LENGTH("사용승인_일") = 8
GROUP BY "건축_구분_코드", "건축_구분_코드_명"
ORDER BY "건축_구분_코드"
"""
result = con_건축.sql(query)
print(result)

┌────────────────┬────────────────────┬─────────┐
│ 건축_구분_코드 │ 건축_구분_코드_명  │  count  │
│    varchar     │      varchar       │  int64  │
├────────────────┼────────────────────┼─────────┤
│ %              │ NULL               │       2 │
│ 0              │ NULL               │       1 │
│ 0100           │ 신축               │ 2211648 │
│ 0200           │ 증축               │  775354 │
│ 0210           │ NULL               │      42 │
│ 0300           │ 개축               │    7870 │
│ 0400           │ 재축               │    4307 │
│ 0500           │ 이전               │    2799 │
│ 0600           │ 대수선             │   35179 │
│ 0700           │ 용도변경           │  160908 │
│ 0710           │ NULL               │      11 │
│ 0800           │ 발코니구조변경     │     537 │
│ 0900           │ NULL               │       2 │
│ 1              │ NULL               │       1 │
│ 2000           │ 허가/신고사항변경  │     571 │
│ 3000           │ 가설건축물축조허가 │    8666 │
│ NULL           │ NULL               │    3281 │
├───

In [170]:
# 최근 10년 사용승인 데이터를 건축행위별로 세기
query = """
SELECT
    "건축_구분_코드",
    "건축_구분_코드_명",
    COUNT(*) AS count
FROM 건축_기본일부
WHERE "사용승인_일" IS NOT NULL
    AND "건축_허가_일" >= '20150101'
    AND "건축_허가_일" <= '20241231'
GROUP BY "건축_구분_코드", "건축_구분_코드_명"
ORDER BY "건축_구분_코드"
"""
result = con_건축.sql(query)
print(result)

┌────────────────┬────────────────────┬────────┐
│ 건축_구분_코드 │ 건축_구분_코드_명  │ count  │
│    varchar     │      varchar       │ int64  │
├────────────────┼────────────────────┼────────┤
│ 0100           │ 신축               │ 781765 │
│ 0200           │ 증축               │ 289701 │
│ 0300           │ 개축               │   2563 │
│ 0400           │ 재축               │   1648 │
│ 0500           │ 이전               │    197 │
│ 0600           │ 대수선             │  19469 │
│ 0700           │ 용도변경           │  33864 │
│ 0800           │ 발코니구조변경     │    433 │
│ 2000           │ 허가/신고사항변경  │     49 │
│ 3000           │ 가설건축물축조허가 │   2011 │
│ NULL           │ NULL               │     66 │
├────────────────┴────────────────────┴────────┤
│ 11 rows                            3 columns │
└──────────────────────────────────────────────┘



최근 10년(2015-2024) 건축인허가 데이터에는 건축 구분 코드 오류가 거의 없음(미기재된 경우 66건)

건축 행정절차별 소요기간은 신축만 집계

In [171]:
# 신축 행위만 추출
query = """
SELECT
    *
FROM 건축_기본일부
WHERE "건축_구분_코드" IN ('0100')
"""
건축_신축 = con_건축.sql(query)
print(건축_신축.limit(5))

# ─── 2. 동별개요, 층별개요 연계 LEFT JOIN ───
query = """
SELECT 
    a.*,
    b."관리_동별_개요_PK",
    b."연면적(㎡)" AS 동별_연면적,
FROM 건축_신축 a
LEFT JOIN 동별개요 b
    ON a."관리_허가대장_PK" = b."관리_허가대장_PK"
"""
건축_join_동별 = con_건축.sql(query)
query = """
SELECT 
    a.*,
    c."관리_층별_개요_PK",
    c."층_구분_코드",
    c."층_구분_코드_명",
    c."주_용도_코드",
    c."주_용도_코드_명",
    c."층_면적(㎡)" AS 층_면적
FROM 건축_join_동별 a
LEFT JOIN 층별개요 c
    ON a."관리_동별_개요_PK" = c."관리_동별_개요_PK"
"""
건축_join_동별층별 = con_건축.sql(query)
print(건축_join_동별층별.limit(5))

┌────────────────────────┬─────────────┬──────────────┬───────────────────┬────────────────┬───────────────────┬───────────────┬──────────────┬──────────────┬──────────────┬──────────────┬─────────────┬─────────────┐
│    관리_허가대장_PK    │ 시군구_코드 │ 주_용도_코드 │  주_용도_코드_명  │ 건축_구분_코드 │ 건축_구분_코드_명 │  연면적(㎡)   │ 건축_허가_일 │ 착공_예정_일 │ 착공_연기_일 │ 실제_착공_일 │ 사용승인_일 │ 추정_착공일 │
│        varchar         │   varchar   │   varchar    │      varchar      │    varchar     │      varchar      │ decimal(18,3) │   varchar    │   varchar    │   varchar    │   varchar    │   varchar   │   varchar   │
├────────────────────────┼─────────────┼──────────────┼───────────────────┼────────────────┼───────────────────┼───────────────┼──────────────┼──────────────┼──────────────┼──────────────┼─────────────┼─────────────┤
│ 1000000000000000001810 │ 28110       │ 01000        │ 단독주택          │ 0100           │ 신축              │       226.100 │ 20220425     │ NULL         │ 20250424     │ NULL         │ NULL        │ NULL

In [172]:
# count the number of records in 건축_신축증축대수선
query = """
SELECT
    COUNT(*) AS count
FROM 건축_신축
WHERE "사용승인_일" >= '20150101'
    AND "사용승인_일" <= '20241231'
"""
result = con_건축.sql(query)
print(result)

┌────────┐
│ count  │
│ int64  │
├────────┤
│ 869205 │
└────────┘



최근 10년 준공, 신축 869,205 건

### 동별개요 검증

In [173]:
# 허가건별 동별 연면적 비교

query = """
WITH summed AS (
    SELECT 
        관리_허가대장_PK,
        건축_구분_코드,
        건축_구분_코드_명,
        MAX("연면적(㎡)") AS 건별_연면적,
        SUM(동별_연면적) AS sum_동별_연면적,
    FROM 건축_join_동별
    GROUP BY 관리_허가대장_PK, "건축_구분_코드", "건축_구분_코드_명"
),
comparison AS (
    SELECT 
        *,
        건별_연면적 = sum_동별_연면적 AS is_equal
    FROM summed
)
SELECT
    건축_구분_코드,
    건축_구분_코드_명,
    COUNT(*) AS count, 
    SUM(CASE WHEN 건별_연면적 IS NULL THEN 1 ELSE 0 END) AS 건별_연면적_null,
    SUM(CASE WHEN sum_동별_연면적 IS NULL THEN 1 ELSE 0 END) AS 동별_연면적_null,
    SUM(CASE WHEN is_equal is NULL THEN 1 ELSE 0 END) AS null_total,
    SUM(CASE WHEN is_equal THEN 1 ELSE 0 END) AS equal,
    SUM(CASE WHEN NOT is_equal THEN 1 ELSE 0 END) AS unequal,
    100.0 * SUM(CASE WHEN is_equal THEN 1 ELSE 0 END) / COUNT(*) AS percentage_equal
FROM comparison
GROUP BY "건축_구분_코드", "건축_구분_코드_명"
ORDER BY "건축_구분_코드"
;
"""
print(con_건축.sql(query))

┌────────────────┬───────────────────┬─────────┬──────────────────┬──────────────────┬────────────┬─────────┬─────────┬───────────────────┐
│ 건축_구분_코드 │ 건축_구분_코드_명 │  count  │ 건별_연면적_null │ 동별_연면적_null │ null_total │  equal  │ unequal │ percentage_equal  │
│    varchar     │      varchar      │  int64  │      int128      │      int128      │   int128   │ int128  │ int128  │      double       │
├────────────────┼───────────────────┼─────────┼──────────────────┼──────────────────┼────────────┼─────────┼─────────┼───────────────────┤
│ 0100           │ 신축              │ 2825419 │                0 │            10502 │      10502 │ 2708289 │  106628 │ 95.85442017626413 │
└────────────────┴───────────────────┴─────────┴──────────────────┴──────────────────┴────────────┴─────────┴─────────┴───────────────────┘



신축의 경우 96%에서 일치하여 동별 연면적 집계가 반드시 필요하지는 않음

### 극단값 검증

In [174]:
# Query to fetch the 연면적 column
query = """
SELECT "연면적(㎡)"
FROM 건축_신축
WHERE "연면적(㎡)" IS NOT NULL
    AND "건축_구분_코드" IN ('0100')
"""

# Fetch the data
area_data = con_건축.sql(query).fetchdf()

# Calculate statistics
max_value = area_data["연면적(㎡)"].max()
mean_value = area_data["연면적(㎡)"].mean()
percentile_1m = area_data["연면적(㎡)"].quantile(0.999999)  # 1 - 1/1,000,000
percentile_100k = area_data["연면적(㎡)"].quantile(0.99999)
percentile_10k = area_data["연면적(㎡)"].quantile(0.9999)
percentile_1k = area_data["연면적(㎡)"].quantile(0.999)  # 1 - 1/1,000
percentile_1h = area_data["연면적(㎡)"].quantile(0.99)  # 1 - 1/100

# Display the results
print(f"Max value:           {max_value: 15.2f}")
print(f"Mean value:          {mean_value: 15.2f}")
print(f"0.999999 percentile: {percentile_1m: 15.2f}")
print(f"0.99999 percentile:  {percentile_100k: 15.2f}")
print(f"0.9999 percentile:   {percentile_10k: 15.2f}")
print(f"0.999 percentile:    {percentile_1k: 15.2f}")
print(f"0.99 percentile:     {percentile_1h: 15.2f}")

Max value:              999999999.00
Mean value:                  1203.71
0.999999 percentile:     16195639.16
0.99999 percentile:       2921348.96
0.9999 percentile:         288728.58
0.999 percentile:           58169.29
0.99 percentile:             7735.27


In [175]:
# Query to fetch data sorted by 연면적 in descending order
query = """
SELECT *
FROM 기본개요
WHERE "건축_구분_코드" IN ('0100')
ORDER BY "연면적(㎡)" DESC
LIMIT 100
"""

# Execute the query and fetch the result
con_건축.sql(query).show(max_rows=10000)
# sorted_data = con_건축.sql(query).fetchdf()

# # Display the result
# with pd.option_context(
#     "display.max_rows",
#     None,
#     "display.max_columns",
#     None,
#     "display.float_format",
#     "{:,.2f}".format,  # Format floats as non-scientific with 2 decimal places
# ):
#     print(sorted_data)

┌────────────────────────┬────────────────────────────────────────────────────┬───────────────────────────────────────────────────────────────────────┬─────────────┬─────────────┬────────────────┬─────────┬─────────┬────────────────────────────────────┬───────────┬─────────┬──────────────┬───────────────────┬───────────────────────┬────────────────────────────┬───────────┬───────────┬───────────┬───────────┬────────────────┬───────────────────┬───────────────┬───────────────┬───────────────┬───────────────┬─────────────────────────┬───────────────┬──────────────┬───────────────────┬──────────────┬────────────────────┬───────────────┬───────────┬───────────────┬────────────┬──────────────┬──────────────┬──────────────┬──────────────┬─────────────┬───────────┐
│    관리_허가대장_PK    │                     대지_위치                      │                                건물_명                                │ 시군구_코드 │ 법정동_코드 │ 대지_구분_코드 │   번    │   지    │             특수지_명              │   블록    │  로

│ 11044660               │ 경기도 고양시 덕양구 고양시덕양구                  │ 장진집입니다잘보세요                                                  │ 41281       │ 00000       │ 0              │ 0000    │ 0000    │ NULL                               │ NULL      │ NULL    │ 과수원       │ NULL              │ NULL                  │ NULL                       │ 03        │ NULL      │ NULL      │ NULL      │ 0100           │ 신축              │ 999999999.000 │ 999999999.000 │       100.000 │ 999999999.000 │           999999999.000 │       100.000 │            1 │                 1 │ NULL         │ NULL               │             5 │         0 │             0 │          0 │ NULL         │ NULL         │ NULL         │ 19980731     │ NULL        │ 20220909  │

가짜 데이터가 들어있음...

│ 1000000000000000452003 │ 부산광역시 강서구 명지동 블록                      │ 명지동 명지지구 복합5블록 복합시설 신축공사                           │ 26440       │ 10400       │ 2              │ 0000    │ 0000    │ 명지지구                           │ 복합5블록 │ NULL    │ 전           │ 도시지역          │ NULL                  │ 지구단위계획구역           │ 01        │ UQA001    │ NULL      │ UQQ300    │ 0100           │ 신축              │     97727.000 │     49700.090 │        50.856 │   1167326.530 │              739058.930 │       756.249 │           11 │                 0 │ 15000        │ 숙박시설           │             0 │      3808 │             0 │          0 │ NULL         │ NULL         │ NULL         │ 20240129     │ NULL        │ 20240129  │

약 10만㎡ 부지에 판매시설(백화점), 근린생활시설 및 숙박시설로 구성된 지하 6층~지상 40층, 연면적 117만1천502㎡ 규모 대규모 복합 쇼핑단지를 건립하는 내용이다.
https://www.yna.co.kr/view/AKR20230307054400051

진짜 사례

오류 사례를 거르기 위하여 개별 사례를 검토 후 연면적이 3,000,000 이상인 경우 (0.001% 미만 사례) 제외하도록 하였다. (건축)

In [176]:
query = """
SELECT
    관리_허가대장_PK,
    시군구_코드,
    주_용도_코드,
    주_용도_코드_명,
    건축_구분_코드,
    건축_구분_코드_명,
    "연면적(㎡)" AS 연면적,
    건축_허가_일 AS 허가일,
    추정_착공일 AS 착공일,
    사용승인_일 AS 준공일,
    datepart('day', (try_strptime(착공일, '%Y%m%d') - try_strptime(허가일, '%Y%m%d')))
    AS 허가착공_기간,
    datepart('day', (try_strptime(준공일, '%Y%m%d') - try_strptime(착공일, '%Y%m%d')))
    AS 착공준공_기간,
    datepart('day', (try_strptime(준공일, '%Y%m%d') - try_strptime(허가일, '%Y%m%d')))
    AS 허가준공_기간
FROM 건축_신축,
WHERE "연면적(㎡)" < 30000000  -- 연면적이 3,000,000㎡ 이하인 경우만
    AND "건축_구분_코드" IN ('0100')  -- 신축 행위만
"""
final_건축 = con_건축.sql(query)
print(final_건축.limit(5))

┌────────────────────────┬─────────────┬──────────────┬───────────────────┬────────────────┬───────────────────┬───────────────┬──────────┬──────────┬──────────┬───────────────┬───────────────┬───────────────┐
│    관리_허가대장_PK    │ 시군구_코드 │ 주_용도_코드 │  주_용도_코드_명  │ 건축_구분_코드 │ 건축_구분_코드_명 │    연면적     │  허가일  │  착공일  │  준공일  │ 허가착공_기간 │ 착공준공_기간 │ 허가준공_기간 │
│        varchar         │   varchar   │   varchar    │      varchar      │    varchar     │      varchar      │ decimal(18,3) │ varchar  │ varchar  │ varchar  │     int64     │     int64     │     int64     │
├────────────────────────┼─────────────┼──────────────┼───────────────────┼────────────────┼───────────────────┼───────────────┼──────────┼──────────┼──────────┼───────────────┼───────────────┼───────────────┤
│ 1000000000000000001810 │ 28110       │ 01000        │ 단독주택          │ 0100           │ 신축              │       226.100 │ 20220425 │ NULL     │ NULL     │          NULL │          NULL │          NULL │
│ 1000000000000000230

In [177]:
# ─── 주택DB: 착공일 추정 컬럼 추가 ───
# 사용_검사_일이 있으면 (착공_일 → 착공_예정_일 → 사용_검사_일), 없으면 착공_일(그대로, null 허용)
query = """
SELECT
    "관리_주택대장_PK",
    "시군구_코드",
    용도_코드,
    용도_코드_명,
    "연면적(㎡)",
    "승인_일",
    "착공_예정_일",
    "착공_일",
    "사용_검사_예정_일",
    "사용_검사_일",
    CASE
        WHEN "사용_검사_일" IS NOT NULL AND "사용_검사_일" != ''
            THEN COALESCE(NULLIF("착공_일", ''), NULLIF("착공_예정_일", ''))
        ELSE "착공_일"
    END AS 추정_착공일
FROM 기본개요;
"""
주택_기본일부 = con_주택.sql(query)
print(주택_기본일부.limit(5))

# 착공_일은 NULL이고 추정_착공일은 NOT NULL인 데이터 5개 추출
query = """
SELECT *
FROM 주택_기본일부
WHERE ("착공_일" IS NULL OR "착공_일" = '')
    AND "추정_착공일" IS NOT NULL
LIMIT 5
"""
result = con_주택.sql(query)
print(result)

┌────────────────────────┬─────────────┬───────────┬──────────────┬───────────────┬──────────┬──────────────┬─────────┬───────────────────┬──────────────┬─────────────┐
│    관리_주택대장_PK    │ 시군구_코드 │ 용도_코드 │ 용도_코드_명 │  연면적(㎡)   │ 승인_일  │ 착공_예정_일 │ 착공_일 │ 사용_검사_예정_일 │ 사용_검사_일 │ 추정_착공일 │
│        varchar         │   varchar   │  varchar  │   varchar    │ decimal(18,3) │ varchar  │   varchar    │ varchar │      varchar      │   varchar    │   varchar   │
├────────────────────────┼─────────────┼───────────┼──────────────┼───────────────┼──────────┼──────────────┼─────────┼───────────────────┼──────────────┼─────────────┤
│ 1000000000000000099018 │ 45210       │ 02000     │ 공동주택     │      7308.110 │ 20230208 │ 20230306     │ NULL    │ 20250317          │ NULL         │ NULL        │
│ 1000000000000000100213 │ 50130       │ 02000     │ 공동주택     │      2196.150 │ 20221223 │ 20221231     │ NULL    │ 20250114          │ NULL         │ NULL        │
│ 1000000000000000121923 │ 11620       │ 02000

In [178]:
query = """
SELECT *
FROM 기본개요
WHERE "관리_주택대장_PK" = '1000000000000000011064'
"""
result = con_주택.sql(query)
print(result)
query = """
SELECT *
FROM 기본개요
WHERE "시군구_코드" = '11110'
    AND "법정동_코드" = '17700'
    AND "대지_구분_코드" = '0'
    AND "번" = '0233'
"""
result = con_주택.sql(query)
print(result)
query = """
SELECT *
FROM 동별개요
WHERE "관리_주택대장_PK" = '1000000000000000011064'
"""
result = con_주택.sql(query)
print(result)

┌────────────────────────┬────────────────────────────────┬─────────┬─────────────┬─────────────┬────────────────┬─────────┬─────────┬───────────┬─────────┬─────────┬───────────┬──────────────┬───────────┬──────────────┬──────────────┬───────────────┬──────────────────┬─────────────────────┬────────────────────────┬──────────────┬──────────────┬──────────────┬──────────┬──────────────┬─────────┬───────────────────┬──────────────┬───────────┐
│    관리_주택대장_PK    │           대지_위치            │ 건물_명 │ 시군구_코드 │ 법정동_코드 │ 대지_구분_코드 │   번    │   지    │ 특수지_명 │  블록   │  로트   │ 용도_코드 │ 용도_코드_명 │ 구조_코드 │ 구조_코드_명 │ 주_건축물_수 │  연면적(㎡)   │ 총_세대_수(세대) │ 철거_멸실_구분_코드 │ 철거_멸실_구분_코드_명 │ 철거_시작_일 │ 철거_종료_일 │ 철거_멸실_일 │ 승인_일  │ 착공_예정_일 │ 착공_일 │ 사용_검사_예정_일 │ 사용_검사_일 │ 생성_일자 │
│        varchar         │            varchar             │ varchar │   varchar   │   varchar   │    varchar     │ varchar │ varchar │  varchar  │ varchar │ varchar │  varchar  │   varchar    │  varchar  │   varchar    │    int32     │ dec

경희궁 자이 3단지: 2017년 준공

2022년에 주택법에 따른 행위가 있음

기본개요 연면적을 사용할 수는 없음

In [179]:
# 행위개요 테이블에서 행위_구분, 행위_구분_코드, 행위_구분_코드_명별 건수 집계
query = """
SELECT
    행위_구분,
    행위_구분_코드,
    행위_구분_코드_명,
    COUNT(*) AS count
FROM 행위개요
WHERE 행위_구분_코드_명 = '신축'
GROUP BY 행위_구분, 행위_구분_코드, 행위_구분_코드_명
ORDER BY 행위_구분, 행위_구분_코드, 행위_구분_코드_명
"""
con_주택.sql(query).show(max_rows=1000)

┌───────────┬────────────────┬───────────────────┬───────┐
│ 행위_구분 │ 행위_구분_코드 │ 행위_구분_코드_명 │ count │
│  varchar  │    varchar     │      varchar      │ int64 │
├───────────┼────────────────┼───────────────────┼───────┤
│ 1         │ 01             │ 신축              │   268 │
│ 2         │ 01             │ 신축              │ 46386 │
│ NULL      │ 01             │ 신축              │    23 │
└───────────┴────────────────┴───────────────────┴───────┘



행위 구분 코드가 01인 경우 신축?

In [180]:
query = """
SELECT *
FROM 행위개요
WHERE "시군구_코드" = '11110'
    AND "법정동_코드" = '17700'
    AND "대지_구분_코드" = '0'
    AND "번" = '0233'
    AND 행위_구분_코드 = '01'  -- 신축 행위만    
"""
result = con_주택.sql(query)
print(result)

query = """
SELECT *
FROM 기본개요
WHERE "관리_주택대장_PK" IN (
    '1002100006686',
    '1002100008687',
)
"""
con_주택.sql(query).show(max_rows=1000)

┌──────────────────┬────────────────────────────────┬─────────────┬─────────────┬────────────────┬─────────┬─────────┬───────────┬─────────┬─────────┬───────────┬────────────────┬───────────────────┬─────────────────┬─────────────┬───────────┬───────────────┬───────────────┬───────────────┬──────────────┬───────────────────────────────────────┬───────────┬───────────────┬────────────┬────────────┬───────────────┬──────────────┬───────────────────┬───────────────┬────────────────────┬────────────────┬───────────────────┬───────────────────┬──────────────────────┬──────────────────┬───────────────────┬──────────────────────┬──────────────────┬────────────────────────┬────────────────────┬────────────────────────┬───────────┐
│ 관리_주택대장_PK │           대지_위치            │ 시군구_코드 │ 법정동_코드 │ 대지_구분_코드 │   번    │   지    │ 특수지_명 │  블록   │  로트   │ 행위_구분 │ 행위_구분_코드 │ 행위_구분_코드_명 │     단지_명     │ 건축물_여부 │ 시설_종류 │ 건축_면적(㎡) │  연면적(㎡)   │ 바닥_면적(㎡) │ 주_용도_코드 │            주_용도_코드_명            │ 기타_용도 │ 공사

기본개요 연면적은 적합하지 않음. 행위개요상 신축이지만 최초 신축이 아닌 행위. 

In [181]:
query = """
SELECT *
FROM 행위개요
WHERE "시군구_코드" = '36110'
    AND "법정동_코드" = '11000'
    AND 행위_구분_코드 = '01'  -- 신축 행위만    
"""
result = con_주택.sql(query)
print(result)

query = """
SELECT *
FROM 기본개요
WHERE "시군구_코드" = '36110'
    AND "법정동_코드" = '11000'
ORDER BY 승인_일
"""
con_주택.sql(query).show(max_rows=1000)

┌────────────────────────┬─────────────────────────────┬─────────────┬─────────────┬────────────────┬─────────┬─────────┬─────────────────────────────┬─────────┬─────────┬───────────┬────────────────┬───────────────────┬────────────────┬─────────────┬───────────┬───────────────┬───────────────┬───────────────┬──────────────┬───────────────────────────────────┬───────────┬───────────────┬────────────┬────────────┬───────────────┬──────────────┬───────────────────┬───────────────┬────────────────────────────────────┬────────────────┬───────────────────┬───────────────────┬──────────────────────┬──────────────────┬───────────────────┬──────────────────────┬──────────────────┬─────────────────────────────┬───────────────────┬─────────────────────────┬───────────┐
│    관리_주택대장_PK    │          대지_위치          │ 시군구_코드 │ 법정동_코드 │ 대지_구분_코드 │   번    │   지    │          특수지_명          │  블록   │  로트   │ 행위_구분 │ 행위_구분_코드 │ 행위_구분_코드_명 │    단지_명     │ 건축물_여부 │ 시설_종류 │ 건축_면적(㎡) │  연면적(㎡)   │ 바닥_면적(㎡)

아파트 단지의 최초 신축은 행위개요에 나타나지 않음.
주택건설 과정에서 주소도 블록 주소에서 도로명 주소로 바뀌기 때문에 주소 기준으로 구분할 수도 없음.

기본개요 용도_코드 부여 여부로 최초 신축을 구분할 수 있음.


In [182]:
query = """
SELECT *
FROM 기본개요
WHERE "시군구_코드" = '36110'
    AND "법정동_코드" = '11000'
    AND 용도_코드 IS NOT NULL
ORDER BY 승인_일
"""
con_주택.sql(query).show(max_rows=1000)

┌──────────────────┬────────────────────────────────┬────────────────┬─────────────┬─────────────┬────────────────┬─────────┬─────────┬─────────────────────────────┬─────────┬─────────┬───────────┬──────────────┬───────────┬──────────────┬──────────────┬───────────────┬──────────────────┬─────────────────────┬────────────────────────┬──────────────┬──────────────┬──────────────┬──────────┬──────────────┬──────────┬───────────────────┬──────────────┬───────────┐
│ 관리_주택대장_PK │           대지_위치            │    건물_명     │ 시군구_코드 │ 법정동_코드 │ 대지_구분_코드 │   번    │   지    │          특수지_명          │  블록   │  로트   │ 용도_코드 │ 용도_코드_명 │ 구조_코드 │ 구조_코드_명 │ 주_건축물_수 │  연면적(㎡)   │ 총_세대_수(세대) │ 철거_멸실_구분_코드 │ 철거_멸실_구분_코드_명 │ 철거_시작_일 │ 철거_종료_일 │ 철거_멸실_일 │ 승인_일  │ 착공_예정_일 │ 착공_일  │ 사용_검사_예정_일 │ 사용_검사_일 │ 생성_일자 │
│     varchar      │            varchar             │    varchar     │   varchar   │   varchar   │    varchar     │ varchar │ varchar │           varchar           │ varchar │ varchar │  varchar  │  

다시 돌아가면...

In [183]:
query = """
SELECT *
FROM 기본개요
WHERE "시군구_코드" = '11110'
    AND "법정동_코드" = '17700'
    AND 용도_코드 IS NOT NULL
"""
result = con_주택.sql(query)
print(result)
query = """
SELECT *
FROM 동별개요
WHERE "관리_주택대장_PK" = '1002100005846'
"""
con_주택.sql(query).show(max_rows=1000)

┌──────────────────┬─────────────────────────────────┬─────────────────┬─────────────┬─────────────┬────────────────┬─────────┬─────────┬───────────┬─────────┬─────────┬───────────┬──────────────┬───────────┬──────────────┬──────────────┬───────────────┬──────────────────┬─────────────────────┬────────────────────────┬──────────────┬──────────────┬──────────────┬──────────┬──────────────┬──────────┬───────────────────┬──────────────┬───────────┐
│ 관리_주택대장_PK │            대지_위치            │     건물_명     │ 시군구_코드 │ 법정동_코드 │ 대지_구분_코드 │   번    │   지    │ 특수지_명 │  블록   │  로트   │ 용도_코드 │ 용도_코드_명 │ 구조_코드 │ 구조_코드_명 │ 주_건축물_수 │  연면적(㎡)   │ 총_세대_수(세대) │ 철거_멸실_구분_코드 │ 철거_멸실_구분_코드_명 │ 철거_시작_일 │ 철거_종료_일 │ 철거_멸실_일 │ 승인_일  │ 착공_예정_일 │ 착공_일  │ 사용_검사_예정_일 │ 사용_검사_일 │ 생성_일자 │
│     varchar      │             varchar             │     varchar     │   varchar   │   varchar   │    varchar     │ varchar │ varchar │  varchar  │ varchar │ varchar │  varchar  │   varchar    │  varchar  │   varchar    │    int3

기본개요 용도 코드로 최초 신축을 구분할 수 있음. 최초 신축 시 기본개요 연면적 사용 가능.

### 극단값 검증

In [184]:
# Query to fetch the 연면적 column
query = """
SELECT "연면적(㎡)"
FROM 기본개요
WHERE "연면적(㎡)" IS NOT NULL
    AND 용도_코드 IS NOT NULL
"""

# Fetch the data
area_data = con_주택.sql(query).fetchdf()

# Calculate statistics
max_value = area_data["연면적(㎡)"].max()
mean_value = area_data["연면적(㎡)"].mean()
percentile_1m = area_data["연면적(㎡)"].quantile(0.999999)  # 1 - 1/1,000,000
percentile_100k = area_data["연면적(㎡)"].quantile(0.99999)
percentile_10k = area_data["연면적(㎡)"].quantile(0.9999)
percentile_1k = area_data["연면적(㎡)"].quantile(0.999)  # 1 - 1/1,000
percentile_1h = area_data["연면적(㎡)"].quantile(0.99)  # 1 - 1/100

# Display the results
print(f"Max value:           {max_value: 15.2f}")
print(f"Mean value:          {mean_value: 15.2f}")
print(f"0.999999 percentile: {percentile_1m: 15.2f}")
print(f"0.99999 percentile:  {percentile_100k: 15.2f}")
print(f"0.9999 percentile:   {percentile_10k: 15.2f}")
print(f"0.999 percentile:    {percentile_1k: 15.2f}")
print(f"0.99 percentile:     {percentile_1h: 15.2f}")

Max value:             5395681227.00
Mean value:                888208.35
0.999999 percentile:   5325231481.43
0.99999 percentile:    4691183771.31
0.9999 percentile:     1388390604.82
0.999 percentile:       107056333.15
0.99 percentile:           438121.01


In [185]:
# Query to fetch data sorted by 연면적 in descending order
query = """
SELECT *
FROM 기본개요
WHERE 용도_코드 IS NOT NULL
ORDER BY "연면적(㎡)" DESC
LIMIT 100
"""

# Execute the query and fetch the result
con_주택.sql(query).show(max_rows=10000)

┌────────────────────────┬────────────────────────────────────────────────┬──────────────────────────────────────┬─────────────┬─────────────┬────────────────┬─────────┬─────────┬──────────────────────────────────────┬─────────┬─────────┬───────────┬───────────────────┬───────────┬──────────────┬──────────────┬────────────────┬──────────────────┬─────────────────────┬────────────────────────┬──────────────┬──────────────┬──────────────┬──────────┬──────────────┬──────────┬───────────────────┬──────────────┬───────────┐
│    관리_주택대장_PK    │                   대지_위치                    │               건물_명                │ 시군구_코드 │ 법정동_코드 │ 대지_구분_코드 │   번    │   지    │              특수지_명               │  블록   │  로트   │ 용도_코드 │   용도_코드_명    │ 구조_코드 │ 구조_코드_명 │ 주_건축물_수 │   연면적(㎡)   │ 총_세대_수(세대) │ 철거_멸실_구분_코드 │ 철거_멸실_구분_코드_명 │ 철거_시작_일 │ 철거_종료_일 │ 철거_멸실_일 │ 승인_일  │ 착공_예정_일 │ 착공_일  │ 사용_검사_예정_일 │ 사용_검사_일 │ 생성_일자 │
│        varchar         │                    varchar                     │      

국내 최대 규모인 올림픽파크 포레온 아파트가 연면적 약 216만 ㎡ 수준이며, 정상적으로 기재되어 있음.

오류 사례를 거르기 위하여 개별 사례를 검토 후 연면적이 3,000,000 이상인 경우 (1% 미만 사례) 제외하도록 하였다. (주택)

In [186]:
# # 신축 필터
# query = """
# SELECT
#     관리_주택대장_PK,
#     시군구_코드,
#     용도_코드 AS 주_용도_코드,
#     용도_코드_명 AS 주_용도_코드_명,
#     "연면적(㎡)" AS 연면적,
#     승인_일 AS 허가일,
#     추정_착공일 AS 착공일,
#     사용_검사_일 AS 준공일
# FROM 주택_기본일부
# """
# 주택_grouped = con_주택.sql(query)
# print(주택_grouped.limit(5))

query = """
SELECT
    관리_주택대장_PK,
    시군구_코드,
    용도_코드 AS 주_용도_코드,
    용도_코드_명 AS 주_용도_코드_명,
    '0100' AS 건축_구분_코드,
    '신축' AS 건축_구분_코드_명,
    "연면적(㎡)" AS 연면적,
    승인_일 AS 허가일,
    추정_착공일 AS 착공일,
    사용_검사_일 AS 준공일,
    datepart('day', (try_strptime(착공일, '%Y%m%d') - try_strptime(허가일, '%Y%m%d')))
    AS 허가착공_기간,
    datepart('day', (try_strptime(준공일, '%Y%m%d') - try_strptime(착공일, '%Y%m%d')))
    AS 착공준공_기간,
    datepart('day', (try_strptime(준공일, '%Y%m%d') - try_strptime(허가일, '%Y%m%d')))
    AS 허가준공_기간
FROM 주택_기본일부
WHERE "연면적(㎡)" IS NOT NULL
    AND 용도_코드 IS NOT NULL  -- 신축만
    AND "연면적(㎡)" < 30000000  -- 연면적이 3,000,000㎡ 이하인 경우만
"""
final_주택 = con_주택.sql(query)
print(final_주택.limit(5))

┌────────────────────────┬─────────────┬──────────────┬─────────────────┬────────────────┬───────────────────┬───────────────┬──────────┬─────────┬─────────┬───────────────┬───────────────┬───────────────┐
│    관리_주택대장_PK    │ 시군구_코드 │ 주_용도_코드 │ 주_용도_코드_명 │ 건축_구분_코드 │ 건축_구분_코드_명 │    연면적     │  허가일  │ 착공일  │ 준공일  │ 허가착공_기간 │ 착공준공_기간 │ 허가준공_기간 │
│        varchar         │   varchar   │   varchar    │     varchar     │    varchar     │      varchar      │ decimal(18,3) │ varchar  │ varchar │ varchar │     int64     │     int64     │     int64     │
├────────────────────────┼─────────────┼──────────────┼─────────────────┼────────────────┼───────────────────┼───────────────┼──────────┼─────────┼─────────┼───────────────┼───────────────┼───────────────┤
│ 1000000000000000099018 │ 45210       │ 02000        │ 공동주택        │ 0100           │ 신축              │      7308.110 │ 20230208 │ NULL    │ NULL    │          NULL │          NULL │          NULL │
│ 1000000000000000100213 │ 50130       │ 

왜 다 착공일 준공일이 없냐...

In [187]:
query = """
SELECT *
FROM 기본개요
WHERE "관리_주택대장_PK" = '1000000000000000107308'

"""
con_주택.sql(query).show(max_rows=1000)
query = """
SELECT *
FROM 기본개요
WHERE "시군구_코드" = '36110'
    AND "법정동_코드" = '11500'
    AND 용도_코드 IS NOT NULL
"""
con_주택.sql(query).show(max_rows=1000)

┌────────────────────────┬────────────────────────────────┬──────────────────────────────────────────────────────┬─────────────┬─────────────┬────────────────┬─────────┬─────────┬───────────┬─────────┬─────────┬───────────┬──────────────┬───────────┬──────────────┬──────────────┬───────────────┬──────────────────┬─────────────────────┬────────────────────────┬──────────────┬──────────────┬──────────────┬──────────┬──────────────┬─────────┬───────────────────┬──────────────┬───────────┐
│    관리_주택대장_PK    │           대지_위치            │                       건물_명                        │ 시군구_코드 │ 법정동_코드 │ 대지_구분_코드 │   번    │   지    │ 특수지_명 │  블록   │  로트   │ 용도_코드 │ 용도_코드_명 │ 구조_코드 │ 구조_코드_명 │ 주_건축물_수 │  연면적(㎡)   │ 총_세대_수(세대) │ 철거_멸실_구분_코드 │ 철거_멸실_구분_코드_명 │ 철거_시작_일 │ 철거_종료_일 │ 철거_멸실_일 │ 승인_일  │ 착공_예정_일 │ 착공_일 │ 사용_검사_예정_일 │ 사용_검사_일 │ 생성_일자 │
│        varchar         │            varchar             │                       varchar                        │   varchar   │   varchar   │    var

다 없는 건 아니군.

In [188]:
query = """
SELECT
    LEFT(준공일, 4) AS 준공_년,
    SUM(연면적)
FROM final_건축
WHERE 준공일 IS NOT NULL
    AND LENGTH(준공일) = 8
    AND 준공일 >= '20150101'
    AND 준공일 <= '20241231'
GROUP BY 준공_년
ORDER BY 준공_년
"""
result = con_건축.sql(query)
print(result)

┌─────────┬───────────────┐
│ 준공_년 │ sum("연면적") │
│ varchar │ decimal(38,3) │
├─────────┼───────────────┤
│ 2015    │  79863416.811 │
│ 2016    │  77108076.741 │
│ 2017    │  81332953.905 │
│ 2018    │  79458251.838 │
│ 2019    │  72122161.041 │
│ 2020    │  89139718.130 │
│ 2021    │  74063304.594 │
│ 2022    │  72271573.510 │
│ 2023    │  67383887.687 │
│ 2024    │  60068625.067 │
├─────────┴───────────────┤
│ 10 rows       2 columns │
└─────────────────────────┘



In [189]:
query = """
SELECT
    LEFT(준공일, 4) AS 준공_년,
    SUM(연면적)
FROM final_주택
WHERE 준공일 IS NOT NULL
    AND LENGTH(준공일) = 8
    AND 준공일 >= '20150101'
    AND 준공일 <= '20241231'
GROUP BY 준공_년
ORDER BY 준공_년
"""
result = con_주택.sql(query)
print(result)

┌─────────┬───────────────┐
│ 준공_년 │ sum("연면적") │
│ varchar │ decimal(38,3) │
├─────────┼───────────────┤
│ 2015    │  33529558.921 │
│ 2016    │  35578227.224 │
│ 2017    │  45794772.809 │
│ 2018    │  56045478.785 │
│ 2019    │  51534192.247 │
│ 2020    │  43534927.676 │
│ 2021    │  38609761.220 │
│ 2022    │  40061585.375 │
│ 2023    │  70902181.017 │
│ 2024    │ 104681360.457 │
├─────────┴───────────────┤
│ 10 rows       2 columns │
└─────────────────────────┘



In [197]:
# Filter and combine 건축 and 주택 data for 신축 행위

query = """
SELECT
    LEFT(착공일, 4) AS 착공_년,
    LEFT(준공일, 4) AS 준공_년,
    LEFT(시군구_코드, 2) AS 시도_코드,
    LEFT(주_용도_코드, 2) AS 용도_대분류_코드,
    연면적,
    허가착공_기간,
    착공준공_기간,
    허가준공_기간
FROM final_건축
WHERE 건축_구분_코드 = '0100'  -- 신축 행위만
"""
df_filtered_건축 = con_건축.sql(query).df()
query = """
SELECT
    LEFT(착공일, 4) AS 착공_년,
    LEFT(준공일, 4) AS 준공_년,
    LEFT(시군구_코드, 2) AS 시도_코드,
    LEFT(주_용도_코드, 2) AS 용도_대분류_코드,
    연면적,
    허가착공_기간,
    착공준공_기간,
    허가준공_기간
FROM final_주택
WHERE 건축_구분_코드 = '0100'  -- 신축 행위만
"""
df_filtered_주택 = con_주택.sql(query).df()
df_filtered_통합 = pd.concat([df_filtered_건축, df_filtered_주택], ignore_index=True)
print(df_filtered_건축.info())
print(df_filtered_주택.info())
print(df_filtered_통합)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2825417 entries, 0 to 2825416
Data columns (total 8 columns):
 #   Column     Dtype  
---  ------     -----  
 0   착공_년       object 
 1   준공_년       object 
 2   시도_코드      object 
 3   용도_대분류_코드  object 
 4   연면적        float64
 5   허가착공_기간    Int64  
 6   착공준공_기간    Int64  
 7   허가준공_기간    Int64  
dtypes: Int64(3), float64(1), object(4)
memory usage: 180.5+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34380 entries, 0 to 34379
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   착공_년       14530 non-null  object 
 1   준공_년       15614 non-null  object 
 2   시도_코드      33620 non-null  object 
 3   용도_대분류_코드  34380 non-null  object 
 4   연면적        34380 non-null  float64
 5   허가착공_기간    14022 non-null  Int64  
 6   착공준공_기간    13717 non-null  Int64  
 7   허가준공_기간    15261 non-null  Int64  
dtypes: Int64(3), float64(1), object(4)
memory usage: 2.2+ MB
None
    

In [211]:
# 전국

## 착공일 기준 연도별 중앙값 산출
df_filtered_착공_10년 = df_filtered_통합[
    (df_filtered_통합["착공_년"] >= "2015") & (df_filtered_통합["착공_년"] <= "2024")
]

# Group by 착공_년 and compute median of 기간 columns
착공_result = (
    df_filtered_착공_10년.groupby("착공_년")[["허가착공_기간"]].median().astype(int)
)
착공_result = 착공_result.sort_index()
착공_result.index.name = "연도"

## 준공일 기준 연도별 중앙값 산출
df_filtered_준공_10년 = df_filtered_통합[
    (df_filtered_통합["준공_년"] >= "2015") & (df_filtered_통합["준공_년"] <= "2024")
]

# Group by 준공_년 and compute median of 기간 columns
준공_result = (
    df_filtered_준공_10년.groupby("준공_년")[["착공준공_기간", "허가준공_기간"]]
    .median()
    .astype(int)
)
준공_result = 준공_result.sort_index()
준공_result.index.name = "연도"

통합_result = pd.merge(
    착공_result,
    준공_result,
    left_index=True,
    right_index=True,
    how="outer",  # use "outer" to preserve all years, or "inner" for common years only
)
display(통합_result)

# Save to Excel file in 'results' directory
output_path = Path("../results/소요기간_전국.csv")
output_path.parent.mkdir(parents=True, exist_ok=True)
통합_result.to_csv(output_path, encoding="utf-8-sig")
print(f"Saved to {output_path}")


,허가착공_기간,착공준공_기간,허가준공_기간
연도,,,
2015,14,162,190
2016,17,178,208
2017,23,191,228
2018,32,196,245
2019,35,196,262
2020,35,191,256
2021,37,196,253
2022,43,217,287
2023,51,251,340


Saved to ..\results\소요기간_전국.csv


In [220]:
# 시도별

## 착공일 기준 연도별 중앙값 산출
df_filtered_착공_10년 = df_filtered_통합[
    (df_filtered_통합["착공_년"] >= "2015") & (df_filtered_통합["착공_년"] <= "2024")
]

# Group by 착공_년 and compute median of 기간 columns
착공_result = (
    df_filtered_착공_10년.groupby(["시도_코드", "착공_년"])[["허가착공_기간"]]
    .median()
    .sort_index()
    .rename_axis(index={"착공_년": "연도"})
    .astype("Int64")  # Nullable int type
)

## 준공일 기준 연도별 중앙값 산출
df_filtered_준공_10년 = df_filtered_통합[
    (df_filtered_통합["준공_년"] >= "2015") & (df_filtered_통합["준공_년"] <= "2024")
]

# Group by 준공_년 and compute median of 기간 columns
준공_result = (
    df_filtered_준공_10년.groupby(["시도_코드", "준공_년"])[
        ["착공준공_기간", "허가준공_기간"]
    ]
    .median()
    .sort_index()
    .rename_axis(index={"준공_년": "연도"})
    .astype("Int64")  # Nullable int type
)

통합_result = pd.merge(
    착공_result,
    준공_result,
    left_index=True,
    right_index=True,
    how="outer",  # use "outer" to preserve all years, or "inner" for common years only
)
display(통합_result)

# Save to Excel file in 'results' directory
output_path = Path("../results/소요기간_시도별.csv")
output_path.parent.mkdir(parents=True, exist_ok=True)
통합_result.to_csv(output_path, encoding="utf-8-sig")
print(f"Saved to {output_path}")


허가착공_기간  착공준공_기간  허가준공_기간
시도_코드 연도                             
11    2015       17      159      183
      2016       27      184      214
      2017       29      205      239
      2018       49      218      273
      2019       63      222      294
...             ...      ...      ...
52    2020       19      152      193
      2021       20      159      198
      2022       23      175      223
      2023       28      196      253
      2024       28      192      250

[174 rows x 3 columns]

Saved to ..\results\소요기간_시도별.csv


In [227]:
# 용도별

## 착공일 기준 연도별 중앙값 산출
df_filtered_착공_10년 = df_filtered_통합[
    (df_filtered_통합["착공_년"] >= "2015")
    & (df_filtered_통합["착공_년"] <= "2024")
    & (df_filtered_통합["용도_대분류_코드"] <= "99")
]

# Group by 착공_년 and compute median of 기간 columns
착공_result = (
    df_filtered_착공_10년.groupby(["용도_대분류_코드", "착공_년"])[["허가착공_기간"]]
    .median()
    .sort_index()
    .rename_axis(index={"착공_년": "연도"})
    .astype("Int64")  # Nullable int type
)

## 준공일 기준 연도별 중앙값 산출
df_filtered_준공_10년 = df_filtered_통합[
    (df_filtered_통합["준공_년"] >= "2015")
    & (df_filtered_통합["준공_년"] <= "2024")
    & (df_filtered_통합["용도_대분류_코드"] <= "99")
]

# Group by 준공_년 and compute median of 기간 columns
준공_result = (
    df_filtered_준공_10년.groupby(["용도_대분류_코드", "준공_년"])[
        ["착공준공_기간", "허가준공_기간"]
    ]
    .median()
    .sort_index()
    .rename_axis(index={"준공_년": "연도"})
    .astype("Int64")  # Nullable int type
)

통합_result = pd.merge(
    착공_result,
    준공_result,
    left_index=True,
    right_index=True,
    how="outer",  # use "outer" to preserve all years, or "inner" for common years only
)
display(통합_result)

# Save to Excel file in 'results' directory
output_path = Path("../results/소요기간_용도별.csv")
output_path.parent.mkdir(parents=True, exist_ok=True)
통합_result.to_csv(output_path, encoding="utf-8-sig")
print(f"Saved to {output_path}")


허가착공_기간  착공준공_기간  허가준공_기간
용도_대분류_코드 연도                             
01        2015       12      168      189
          2016       13      182      204
          2017       20      195      225
          2018       29      204      248
          2019       30      205      269
...                 ...      ...      ...
31        2023       52      251      319
          2024       49      278      369
33        2022       36     <NA>     <NA>
          2023       66      523      666
          2024      100      741      770

[309 rows x 3 columns]

Saved to ..\results\소요기간_용도별.csv


In [35]:
# 준공일 기준 집계

# 건축
query = """
SELECT
    LEFT(준공일, 6) AS 준공_년월,
    LEFT(시군구_코드, 2) AS 시도_코드,
    LEFT(주_용도_코드, 2) AS 용도_대분류_코드,
    건축_구분_코드,
    건축_구분_코드_명,
    SUM(연면적)
FROM final_건축
WHERE 준공일 IS NOT NULL
    AND LENGTH(준공일) = 8
    AND 준공일 >= '20150101'
    AND 준공일 <= '20241231'
GROUP BY LEFT(준공일, 6), 
    LEFT(시군구_코드, 2),
    LEFT(주_용도_코드, 2),
    건축_구분_코드,
    건축_구분_코드_명,
ORDER BY LEFT(준공일, 6)
"""
df_건축_10년_준공 = con_건축.sql(query).df()
print(df_건축_10년_준공)

# 주택
query = """
SELECT
    LEFT(준공일, 6) AS 준공_년월,
    LEFT(시군구_코드, 2) AS 시도_코드,
    LEFT(주_용도_코드, 2) AS 용도_대분류_코드,
    건축_구분_코드,
    건축_구분_코드_명,
    SUM(연면적)
FROM final_주택
WHERE 준공일 IS NOT NULL
    AND LENGTH(준공일) = 8
    AND 준공일 >= '20150101'
    AND 준공일 <= '20241231'
GROUP BY LEFT(준공일, 6), 
    LEFT(시군구_코드, 2),
    LEFT(주_용도_코드, 2),
    건축_구분_코드,
    건축_구분_코드_명,
ORDER BY LEFT(준공일, 6)
"""
df_주택_10년_준공 = con_주택.sql(query).df()
print(df_주택_10년_준공)

        준공_년월 시도_코드 용도_대분류_코드 건축_구분_코드 건축_구분_코드_명  sum("연면적")
0      201501    41        03     0400         재축      90.000
1      201501    31        04     0100         신축   12459.320
2      201501    26        04     0200         증축    3554.858
3      201501    29        11     0100         신축     500.130
4      201501    28        18     0100         신축    2093.300
...       ...   ...       ...      ...        ...         ...
76978  202412    26        04     0100         신축   15062.912
76979  202412    46        27     0200         증축     222.870
76980  202412    48        18     0200         증축    2936.575
76981  202412    30        06     0200         증축      28.840
76982  202412    31        01     0600        대수선     202.500

[76983 rows x 6 columns]
       준공_년월 시도_코드 용도_대분류_코드 건축_구분_코드 건축_구분_코드_명  sum("연면적")
0     201501    50        03     0100         신축     359.522
1     201501    46        04     0100         신축     569.060
2     201501    11        03     0100         신

In [36]:
# Concatenate the two DataFrames
df_준공_10년_통합 = pd.concat([df_건축_10년_준공, df_주택_10년_준공], ignore_index=True)

# Save to Excel file in 'results' directory
output_path = Path("../results/준공_10년_통합.xlsx")
output_path.parent.mkdir(parents=True, exist_ok=True)
df_준공_10년_통합.to_excel(output_path, index=False)
print(f"Saved to {output_path}")

Saved to ..\results\준공_10년_통합.xlsx
